In [0]:
%pip install nltk

Python interpreter will be restarted.
Collecting nltk
 Using cached nltk-3.6.5-py3-none-any.whl (1.5 MB)
Collecting tqdm
 Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
Collecting click
 Using cached click-8.0.3-py3-none-any.whl (97 kB)
Collecting regex>=2021.8.3
 Using cached regex-2021.10.8-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (764 kB)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.8/site-packages (from nltk) (1.0.1)
Installing collected packages: tqdm, regex, click, nltk
Successfully installed click-8.0.3 nltk-3.6.5 regex-2021.10.8 tqdm-4.62.3
WARNING: You are using pip version 21.0.1; however, version 21.3 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-f85666fa-0f34-4c6a-b2f6-e3650ee6f7cb/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
# Stopwords lists
import requests
stopwords_list = requests.get("https://gist.githubusercontent.com/rg089/35e00abf8941d72d419224cfd5b5925d/raw/12d899b70156fd0041fa9778d657330b024b959c/stopwords.txt").content
stopwords_2 = set(stopwords_list.decode().splitlines()) 

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!

In [0]:
data_output = spark.read.option("header","true").parquet("dbfs:/mnt/group12/sentiment")

In [0]:
import re
# Clean text udf
def clean_body(x):
    punc='!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~'
    cleaned = x.lower()
    cleaned = re.sub(r'https?:\/\/.*[\r\n]*', '', cleaned, flags=re.MULTILINE)
    cleaned = re.sub(r'[u|a]\d+.*', '', cleaned, flags=re.MULTILINE) # remove special line break characters
    for ch in punc:
        cleaned = cleaned.replace(ch, '')
    return cleaned

clean_body_udf = F.udf(clean_body , StringType())


In [0]:
StopWords = stopwords.words("english")
# Filter tokens udf
def stop_words_filter(x):
    return (~x.isin(StopWords)) & (~x.isin(stopwords_2)) & (x.isNotNull()) & (F.length(x) > 2)

data_chunk_tokens = data_output.withColumn('cleaned_body', clean_body_udf(F.col('body')))
data_chunk_tokens = data_chunk_tokens.withColumn('tokens', F.filter(F.split(F.col('cleaned_body'), ' '), stop_words_filter))

In [0]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

# Word ocurrences count UDF
def word_count_agg(token_lists):
    word_count = {}
    for token_list in token_lists:
    for token in token_list:
        if token not in word_count:
            word_count[token] = 0
            word_count[token] += 1
    sorted_word_count = dict(sorted(word_count.items(), key=lambda item: item[1], reverse=True))
    n_items = take(50, sorted_word_count.items()) #Top 50
    return dict(n_items)

udf_word_count_agg = F.udf(word_count_agg , MapType(StringType(), IntegerType()))

In [0]:
# Group by topic and aggregate metrics
data_grouped = data_chunk_tokens.filter(F.col('topic_title').isNotNull()).groupby("created_at_month", "topic_title").agg(
    F.count("id").alias("number_of_messages"),
	F.sum("like_count").alias("likes_total"), 
	F.avg("like_count").alias("average_likes"),  
	F.sum("dislike_count").alias("dislike_total"), 
	F.avg("dislike_count").alias("average_dislike"),
	F.sum("score").alias("score_total"), 
    F.avg("score").alias("average_score"), 
	F.avg("vader_sentiment").alias("avg_vader_sentiment"), 
    F.create_map(
        F.lit('negative_emotions'), F.avg("liwc_sentiment.negemo"),
        F.lit('positive_emotions'), F.avg("liwc_sentiment.posemo"),
        F.lit('anger'), F.avg("liwc_sentiment.anger"), 
        F.lit('sad'), F.avg("liwc_sentiment.sad"), 
        F.lit('money'), F.avg("liwc_sentiment.money"),	
        F.lit('health'), F.avg("liwc_sentiment.health"), 
        F.lit('social'), F.avg("liwc_sentiment.social"),
        F.lit('anxiety'), F.avg("liwc_sentiment.anx"), 
        F.lit('humans'), F.avg("liwc_sentiment.friends"), 
        F.lit('family'), F.avg("liwc_sentiment.family"), 
        F.lit('friends'), F.avg("liwc_sentiment.friends"), 
        F.lit('focusfuture'), F.avg("liwc_sentiment.focusfuture"), 
        F.lit('focuspast'), F.avg("liwc_sentiment.focuspast"), 
        F.lit('focuspresent'), F.avg("liwc_sentiment.focuspresent"), 
        F.lit('work'), F.avg("liwc_sentiment.work"),
        F.lit('drives'), F.avg("liwc_sentiment.drives"), 
        F.lit('discrepancies'), F.avg("liwc_sentiment.discrep"),
        F.lit('time'), F.avg("liwc_sentiment.time"), 
        F.lit('leisure'), F.avg("liwc_sentiment.leisure"),
        F.lit('death'), F.avg("liwc_sentiment.death"), 
        F.lit('religion'), F.avg("liwc_sentiment.relig"),
    ).alias('liwc_sentiment_map'),
    udf_word_count_agg(F.collect_list("tokens")).alias('word_count'),
    F.slice(F.collect_list(F.col('body')), 1, 20).alias('messages_sample')
)

In [0]:
data_grouped.count()

Out[18]: 8416

In [0]:
data_grouped.show()

+----------------+--------------------+------------------+-----------+------------------+-------------+--------------------+-----------+------------------+--------------------+--------------------+--------------------+--------------------+
created_at_month| topic_title|number_of_messages|likes_total| average_likes|dislike_total| average_dislike|score_total| average_score| avg_vader_sentiment| liwc_sentiment_map| word_count| messages_sample|
+----------------+--------------------+------------------+-----------+------------------+-------------+--------------------+-----------+------------------+--------------------+--------------------+--------------------+--------------------+
 2017-05| Health Care Bill | 259| 2171| 8.382239382239382| 57| 0.22007722007722008| 2114| 8.162162162162161| 0.05426988409810545|{negative_emotion...|{insurance -> 22,...|[Check out www.di...|
 2017-05| Julian Assange| 118| 1548| 13.11864406779661| 71| 0.6016949152542372| 1477|12.516949152542374|0.010166947650959936|{negative_emotion...|{reason -> 5, usa...|[there is no frea...|
 2017-05| Macron| 47| 600| 12.76595744680851| 14| 0.2978723404255319| 586| 12.46808510638298| 0.0225085104875108|{negative_emotion...|{trending -> 2, g...|[What I said and ...|
 2017-05| Roger Ailes| 75| 396| 5.28| 49| 0.6533333333333333| 344| 4.586666666666667|-0.00445599973201...|{negative_emotion...|{die -> 2, smith ...|[you mean like re...|
 2017-05| Seth Rich| 3339| 26563| 7.955375861036239| 1705| 0.5106319257262654| 24818| 7.432764300688829|0.003420454777337...|{negative_emotion...|{trump -> 169, ev...|[The head doc at ...|
 2017-05| WhatsApp Down| 3| 38|12.666666666666666| 0| 0.0| 38|12.666666666666666| 0.0|{negative_emotion...|{whatsapp -> 1, l...|[Iphones shorten ...|
 2017-06| Alexandria| 1528| 14732| 9.641361256544503| 76|0.049738219895287955| 14656| 9.591623036649215| -0.1877498687319616|{negative_emotion...|{violent -> 39, g...|[Oh that's right ...|
 2017-06| Bill Cosby| 42| 371| 8.833333333333334| 11| 0.2619047619047619| 361| 8.595238095238095|-0.24214761788468986|{negative_emotion...|{big -> 2, dig ->...|[\"The defense re...|
 2017-06| Father's Day| 267| 3553|13.307116104868914| 18| 0.06741573033707865| 3535|13.239700374531836| 0.2710303374276625|{negative_emotion...|{big -> 10, trump...|[Lena Dunham is H...|
 2017-06| Kathy Griffin| 22| 155| 7.045454545454546| 5| 0.22727272727272727| 150| 6.818181818181818|-0.01672272621230...|{negative_emotion...|{neverybody -> 1,...|[There was only o...|
 2017-06|Notre Dame Cathedral| 65| 555| 8.538461538461538| 44| 0.676923076923077| 511| 7.861538461538461|-0.07012923147815925|{negative_emotion...|{london -> 5, not...|[SHOCKING VIDEO e...|
 2017-06| The Vatican | 22| 75| 3.409090909090909| 13| 0.5909090909090909| 62|2.8181818181818183| 0.1582090895284306|{negative_emotion...|{pope -> 5, big -...|[So then we must ...|
 2017-07| AlphaBay| 155| 665| 4.290322580645161| 8| 0.05161290322580645| 657|4.2387096774193544|0.014033548509882343|{negative_emotion...|{trump -> 9, abor...|[Christian evange...|
 2017-07| Imran Awan| 2| 2| 1.0| 0| 0.0| 2| 1.0| 0.0|{negative_emotion...|{crumble -> 1, ca...|[This could be wh...|
 2017-07| Is It Fake News?| 12| 27| 2.25| 0| 0.0| 27| 2.25| -0.2507583334421118|{negative_emotion...|{wanting -> 1, tr...|[MSNBC\u2019s \u2...|
 2017-07| Kid Rock Senate| 55| 412| 7.490909090909091| 2| 0.03636363636363636| 410| 7.454545454545454| 0.13043999834494158|{negative_emotion...|{reason -> 2, die...|[agreed. Here is ...|
 2017-07| Mt Gox | 2| 4| 2.0| 0| 0.0| 4| 2.0| -0.3960500117391348|{negative_emotion...|{activity -> 1, c...|[Many years ago, ...|
 2017-08| #Alt-Left Terrorism| 26| 241| 9.26923076923077| 0| 0.0| 241| 9.26923076923077|-0.15994615508959845|{negative_emotion...|{couple -> 1, ago...|[Never too early ...|
 2017-08| #GoodNews| 2| 2| 1.0| 0| 0.0| 2| 1.0| 0.8240500092506409|{negative_emotion...|{love -> 1, weeks...|[Lol, promise I w...|
 2017-08| ANTIFA Petition| 98| 1612|16.448979591836736| 1| 0.010204081

In [0]:
# Order by score
data_ordered = data_grouped.orderBy(F.col('created_at_month').asc(), F.col('score_total').desc())

In [0]:
# Getting top 5 users for the topics
data_user = data_chunk_tokens.filter(F.col('topic_title').isNotNull()).groupby("created_at_month", "topic_title","username").agg(F.sum("score").alias("User_score"))
User_order = data_user.orderBy(F.col('created_at_month').asc(),F.col('topic_title'), F.col('User_score').desc())
a = data_ordered.alias("dO").join(User_order.alias("UO"),
                          ((F.col("dO.created_at_month") == F.col("UO.created_at_month")) & (F.col("dO.topic_title") == F.col("UO.topic_title"))),
                               "inner")
n = a.orderBy(F.col('dO.created_at_month').asc(),F.col('dO.topic_title'),F.col('User_score').desc())

In [0]:
B = n.groupby("dO.created_at_month", "dO.topic_title").agg(
  F.slice(F.collect_list(F.col('username')), 1, 5).alias('users'),
  F.first('number_of_messages').alias('number_of_messages'),
  F.first('likes_total').alias('likes_total'),
  F.first('average_likes').alias('average_likes'),
  F.first('dislike_total').alias('dislike_total'),
  F.first('average_dislike').alias('average_dislike'),
  F.first('score_total').alias('score_total'),
  F.first('average_score').alias('average_score'),
  F.first('avg_vader_sentiment').alias('avg_vader_sentiment'),
  F.first('liwc_sentiment_map').alias('liwc_sentiment_map'),
  F.first('word_count').alias('word_count'),
  F.first('messages_sample').alias('messages_sample')
)

In [0]:
# Filter top 10 topics per period
from pyspark.sql.window import *
data_ordered_with_n = B.withColumn("row_num", F.row_number().over(Window.partitionBy("created_at_month").orderBy(F.col('score_total').desc())))
data_ordered_with_n = data_ordered_with_n.filter(F.col('row_num') < 10)
data_ordered_with_n_ordered = data_ordered_with_n.orderBy(F.col('created_at_month').asc(), F.col('score_total').desc())

In [0]:
# Write top 10 output 
data_ordered_with_n_ordered.coalesce(1).write.mode('overwrite').json("dbfs:/mnt/group12/data_output/topics_top_10.json")

In [0]:
# Write output
B.coalesce(1).write.mode('overwrite').json("dbfs:/mnt/group12/data_output/topics.json")